# The aim of this notebook is to implement the work presented in the article [here](https://towardsdatascience.com/how-to-train-multiple-machine-learning-models-and-run-other-data-tasks-in-parallel-by-combining-2fa9670dd579)

## Reading in packages

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import time
import h5py
import copy
import datetime
import ta
import pathlib
import shutil
import tempfile
# import vaex
from IPython import display
from IPython.display import clear_output
import pyodbc

# Tensorflow related
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers
from tensorflow.keras import regularizers
import tensorflow.compat.v2.feature_column as fc

#!pip install -q git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

print(tf.__version__)
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)
print(logdir)

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score, f1_score, log_loss


# Models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning 
from sklearn import ensemble
# ConvergenceWarning('ignore')
# Do you wanna see?
verbose = True

import sys
sys.path.append('../')
#sys.path.append('...../')

from utils.data_extraction import load_data_final,load_data_and_save
from utils.data_cleaning import HFDataCleaning
from utils.generate_features import candleCreateNP_vect_final,\
                                    generateFeatures_final,\
                                    generateFeatures_multi_v2

from utils.preprocessing_features_and_labels import extract_labels,\
                                                    align_features_and_labels,\
                                                    pre_processing_initial,\
                                                    pre_processing_extended,\
                                                    pre_processing,\
                                                    extract_labels_multi_final,\
                                                    align_features_and_labels_multi_final,\
                                                    align_features_and_labels_multi_v5

from utils.models import make_input_fn
from utils.models import performanceTesting,scoreFunction
from utils.plotting import plot_confusion_matrix

2.2.0
C:\Users\fstri\AppData\Local\Temp\tmpshdx2d3y\tensorboard_logs


In [33]:
test_data = list(range(11))
test_data

#pd.cut(test_data, bins=5)
#.qcut(test_data, )

# xyz, splits = pd.qcut(test_data, q=[0, 0.2, 0.8, 1], labels=False, retbins=True)
xyz, splits = pd.qcut(test_data, q=[0, 0.1, 0.3, 0.7, 0.9, 1], labels=False, retbins=True)

print(xyz, splits)

labels = pd.cut(test_data, bins=splits, labels=False, right=False, include_lowest=True)
labels

[0 0 1 1 2 2 2 2 3 3 4] [ 0  1  3  7  9 10]


array([ 0.,  1.,  1.,  2.,  2.,  2.,  2.,  3.,  3.,  4., nan])

In [34]:
 pd.qcut(test_data, q=[0, 0.2, 0.8, 1], labels=False) #, retbins=True)

array([0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2], dtype=int64)

## Extracting data

In [36]:
# Do we extract new data or read in?
readIn = True
# run load_data()
if readIn:
    
    # Listing the data files 
    #path = '../../../Google Drev/Thesis/Data/TAQ/AggregatedTAQ'
    path = 'F:/AggregatedTAQ/round3'
    datafiles = os.listdir(path)
    content = np.concatenate([['\n\n'],[str(j)+': '+i+'\n' for j,i in enumerate(datafiles) if 'csv' in i],['\n\n']])
    
    # Asking for user input
    file = input('Which one do you want to load? %s'%''.join(content))
    data = pd.read_csv(path + '/' + datafiles[int(file)],
                       header = None,
                       names=['open','high','low','close',
                              'spread_open','spread_high','spread_low','spread_close',
                              'bidsize_open','bidsize_high','bidsize_low','bidsize_close',
                              'ofrsize_open','ofrsize_high','ofrsize_low','ofrsize_close',
                              'Ticker'])
    # Lower casing all column names
#     data.columns = data.columns.str.lower()
else:
    
    # print(os.listdir())
    try:
        path = 'a:/taqhdf5'  #'a:/taqhdf5'
        os.listdir(path)
    except:
        path = 't:/taqhdf5'  #'a:/taqhdf5'
        os.listdir(path)
        
    # Sample type
    data_sample = 'full' # or 'stable'
    # allFiles = os.listdir(path)
    # print(len(allFiles), allFiles[:5], allFiles[-5:])
    # print(allFiles[-10:])

    #dates = np.array(['2020040' + str(i) if i < 10 else '202004' + str(i) for i in np.arange(1,16)]).astype(int)
    dates = np.array(['20200501']).astype(int)#,'20200402','20200403','20200406','20200407'

    # Provide a list of tickers of interest
    
    tickers = sorted(['TSLA','FB'])#'MSFT'
    
    # Do we need data on trades, quotes or both?
    dataNeeded = 'quotes' # 'trades', 'quotes' or 'both'
    
    if dataNeeded == 'trades':
        tradeData = load_data_final(dates, tickers, dataNeeded, path, verbose)
    elif dataNeeded == 'quotes':
        quoteData = load_data_final(dates,
                                    tickers,
                                    dataNeeded,
                                    path,
                                    verbose,
                                    extract_candles = False,
                                    aggHorizon = 1,
                                    extra_features_from_quotes = None,
                                    data_sample = data_sample)
    elif dataNeeded == 'both':
        tradeData, quoteData = load_data_final(dates, tickers, dataNeeded, path, verbose)

# Reading in sector information
stockInfo = pd.read_csv('../utils/stockInfo_v1.csv',header=[0,1])
stockInfo.columns = ['ticker','sector','exchange','marketCap']

# Creating a table with stock information based on the tickers available in the data.
uniqueTickers = data.Ticker.unique()
stockTable = stockInfo[stockInfo.ticker.isin(uniqueTickers)]
stockTable.head(10)

Which one do you want to load? 

0: aggregateTAQ_10sec.csv
1: aggregateTAQ_30sec.csv
2: aggregateTAQ_60sec.csv


2


,ticker,sector,exchange,marketCap
12,AAPL,Technology,NMS,1.578173e+12
20,ABBV,Healthcare,NYQ,1.742612e+11
34,ABT,Healthcare,NYQ,1.631410e+11
126,AEP,Utilities,NYQ,4.089551e+10
379,AMT,Real Estate,NYQ,1.171259e+11
428,APD,Basic Materials,NYQ,5.464395e+10
697,BA,Industrials,NYQ,1.020356e+11
699,BABA,Consumer Cyclical,NYQ,5.936536e+11
700,BAC,Financial Services,NYQ,2.020550e+11
870,BHP,Basic Materials,NYQ,1.258194e+11


### Dropping ETFS

In [37]:
etfs = ['IYH','IYM','IYK','IYJ','IYG','IYW','IYC','IYR','IDU','IYZ','IYE','IYF']

# Extracting the sector ETFs to a separate variable
sectorETFS = data[data.Ticker.isin(etfs)]

# Removing the ETFs
data = data[~data.Ticker.isin(etfs)]

## Generating Features

In [44]:
########### Generate Features ################

n_feature_lags = 1

features = generateFeatures_multi_v2(data = data, 
                                  listOfFeatures = [
                                                    'pastobs',
                                                    'spread',
                                                    'bidsize',
                                                    'ofrsize',
#                                                     'stok',
#                                                     'stod',
#                                                     'sstod',
#                                                     'wilr',
#                                                     'roc',
#                                                     'rsi',
#                                                     'atr',
#                                                     'cci',
#                                                     'dpo',
#                                                     'sma',
#                                                     'ema',
#                                                     'macd',
#                                                       'macd_diff',
#                                                       'macd_signal',
#                                                     'dis5',
#                                                     'dis10',
                                                      'sector'
                                                   ], 
                                   feature_lags = n_feature_lags
                                     ,stockTable=stockTable)

########### Generate Labels ################

n_classes = 2
# extract first 4 columns as the lag0 or raw OHLC prices (used for labelling)
price_candles = data[['open','high','low','close','Ticker']]

########### Align Data ################

# from imported function (see testing_preprocessing_features_and_labels.ipynb for thorough experimenting with all the cut-offs):    
X, y = align_features_and_labels_multi_final(price_candles = price_candles, 
                                             all_features = features,
                                             prediction_horizon = 1, 
                                             n_feature_lags = n_feature_lags, 
                                             n_classes = n_classes, # 5,
                                             safe_burn_in = False, 
                                             data_sample = 'full',
                                             splitType='global',
                                             noise=False,ticker_dummies=False)

AAPL done
ABBV done
ABT done
AEP done
AMT done
APD done
BA done
BABA done
BAC done
BHP done
BP done
CCI done
CHL done
COST done
CSGP done
D done
DIS done
ECL done
ENB done
EXC done
FB done
FMX done
GOOG done
INTC done
JNJ done
KO done
LFC done
LIN done
LMT done
MA done
MCD done
MSFT done
NKE done
NVDA done
NVS done
Number of NaNs in label: 1. 1 is expected
Returns that lead to NaNs in label: [0.0907158]
PBR done
PEP done
PFE done
PLD done
PSA done
PTR done
PYPL done
RTX done
SHW done
SNP done
SO done
SRE done
T done
TM done
TSLA done
TSM done
UNP done
UPS done
V done
WMT done


# Expanding functions:

- extract_labels_multi 
- align_features_and_labels_multi

In [39]:
## Works only for two classes (by using median)
# def extract_labels_multi(data = None,
#                         classes = 5,
#                         group_style = 'equal',
#                         splits=None):
    
#     # this version takes data in a direct returns for a specific ticker

#     if group_style == 'equal':
#         # if splits is None:
#             # splits = np.array_split(np.sort(returns),classes)

#         # for i in np.arange(classes):

#         #labels[returns > global_median] = 1
#         #labels[returns <= global_median] = 0
        
#         labels = pd.cut(data, bins=splits, labels=False, right=False, include_lowest=True)
        
#         # we need right=False (open right-handside in split interval) to get median into the positive class

#     elif group_style != 'equal':
#         raise ValueError(f'group_style {group_style} not implemented')

#     return labels #, returns, [thresholdsMin, thresholdsMax]

# per version 6 we no longer use group_style, as the "splits" fully describes splits for both equal and non-equal
def extract_labels_multi_v6(data = None,
                            classes = 5,
                            splits=None):

    # this version takes data in a direct returns for a specific ticker
    # per version 6 we no longer use group_style, as the "splits" fully describes splits for both equal and non-equal

    labels = pd.cut(data, bins=splits, labels=False, right=False, include_lowest=True)

    # we need right=False (open right-handside in split interval) to get median into the positive class
    # this makes the last point nan, we fix it here
    if sum(np.isnan(labels)) > 0:
        print(f'Number of NaNs in label: {sum(np.isnan(labels))}. 1 is expected')
        print(f'Returns that lead to NaNs in label: {data[np.where(np.isnan(labels))]}')
        assert sum(np.isnan(labels)) <= 1, "There should be max 1 NaN"

        if data[np.where(np.isnan(labels))] >= splits[-1]:
            labels[np.where(np.isnan(labels))] = classes - 1 # assign last label id
        else:
            print(data[np.where(np.isnan(labels))], splits[-1])
            raise ValueError('There is a label NaN where its underlying return is not max of dataset, which it should be')

    return labels


# def align_features_and_labels_multi(price_candles,
#                                     all_features,
#                                     prediction_horizon,
#                                     n_feature_lags,
#                                     n_classes,
#                                     safe_burn_in = False,
#                                     data_sample = 'full',
#                                     splitType='global',
#                                     noise = True):

#     all_burned_in_features = pd.DataFrame()
#     all_labels = pd.DataFrame()
    
#     if splitType.lower() == 'global':
#         # Making the splits for the labels based on all tickers
#         # returns = ((price_candles['close'].values[1:] / price_candles['close'].values[:-1]) -1) * 100
# #         returns = np.concatenate([((price_candles[price_candles.Ticker==ticker]['close'].values[1:]/\
# #                          price_candles[price_candles.Ticker==ticker]['close'].values[:-1])-1) for ticker\
# #                           in price_candles.Ticker.unique()])

#         returns = []
#         tickers = []
#         for ticker in price_candles.Ticker.unique():

#             ticker_returns = (price_candles[price_candles.Ticker==ticker]['close'].values[1:]/\
#                                  price_candles[price_candles.Ticker==ticker]['close'].values[:-1]) - 1
#             ticker_names = [ticker for i in range(len(ticker_returns))]

#             returns.append(ticker_returns)
#             tickers.append(ticker_names)

#         # concatenate returns and add noise    
#         returns = np.concatenate(returns)
#         if noise:
#             returns[returns==0] = np.random.normal(0,1,sum(returns==0))/1000000

#         tickers = np.concatenate(tickers)

#         _, splits = pd.qcut(returns, q=n_classes, labels=False, retbins=True)
#         print(splits)
        
#         returns = pd.DataFrame({'returns': returns, 'Ticker': tickers})
        
        
        
#     for ticker_iter, ticker_name in enumerate(all_features.ticker.unique()):
#         ticker_features = all_features[all_features.ticker==ticker_name].copy(deep=True)
#         # removing the "ticker" variable from ticker_features as np.isnan() does not like non-numericals
#         #ticker_features = ticker_features.iloc[:, ticker_features.columns != 'ticker']
#         ticker_features.drop('ticker', axis=1, inplace=True)
#         # extract first 4 columns as the lag0 or raw OHLC prices (used for labelling)
#         #ticker_prices = price_candles[price_candles.Ticker==ticker_name]['close'].values # candles.iloc[:, :4].values
#         ticker_returns = returns[returns.Ticker==ticker_name]['returns'].values

#         if not safe_burn_in:
#             assert data_sample == 'full'
#             # we assume data_sample is full and that we can continue features from yesterday's values.
#             # that we have a single burn-in at the beginning and that's it

#             # get first index that has no NaNs (the sum checks for True across columns, we look for sum == 0 and where that is first True)
#             burned_in_idx = np.where((np.sum(np.isnan(ticker_features.values), axis=1) == 0) == True)[0][0]

#             # calculate end-point cut-off to match with labels
#             end_point_cut = max(prediction_horizon, n_feature_lags + 1)

#             # slice away the observations used for burn-in (taking off 1 at the end to match with labels [slice off "prediction_horizon"])
#             burned_in_features = ticker_features.iloc[burned_in_idx : -end_point_cut, :] #.reset_index(drop=True) # features[burned_in_idx:] latter is sligthly faster but maybe not as precise

#             # slice away the burned-in indices from labels
#             labels = extract_labels_multi(data = ticker_returns[(burned_in_idx+n_feature_lags):],
#                                           classes = n_classes,
#                                           group_style = 'equal',
#                                           splits = splits)
#             # labels, returns, thresholds = extract_labels(data = candles[burned_in_idx + n_feature_lags : , :],
#             #                                             classes = n_classes, group_style = 'equal')

#             # check if there are remaining NaNs are burn-in (means error)
#             remaining_nans = np.where(np.isnan(burned_in_features.values))[0].size
#             if remaining_nans > 0:
#                 raise ValueError('Had NaN in burned_in_features after burn-in')

#         burned_in_features['ticker'] = ticker_name
#         all_burned_in_features = pd.concat([all_burned_in_features, burned_in_features])
#         all_labels = pd.concat([all_labels, pd.Series(labels)])
#         print(ticker_name + " done")

#     return all_burned_in_features, all_labels.reset_index(drop=True) # call the function as X, y = align_features_and_labels(.) if you like


# adding custom label splitting (label_split) for multi-class
def align_features_and_labels_multi_v8(price_candles,
                                        all_features,
                                        prediction_horizon,
                                        n_feature_lags,
                                        n_classes,
                                        label_split = [],
                                        safe_burn_in = False,
                                        data_sample = 'full',
                                        splitType='global',
                                        noise = False,
                                        ticker_dummies = False):

    all_burned_in_features = pd.DataFrame()
    all_burned_in_indices = pd.DataFrame()
    all_labels = pd.DataFrame()

    dailyIndices = pd.DataFrame({'days':price_candles.index.get_level_values(0),
                                  'timestamps':price_candles.index.get_level_values(1),
                                  'ticker':price_candles.Ticker})

    if splitType.lower() == 'global':
        # Making the splits for the labels based on all tickers
        # returns = ((price_candles['close'].values[1:] / price_candles['close'].values[:-1]) -1) * 100
#         returns = np.concatenate([((price_candles[price_candles.Ticker==ticker]['close'].values[1:]/\
#                          price_candles[price_candles.Ticker==ticker]['close'].values[:-1])-1) for ticker\
#                           in price_candles.Ticker.unique()])

        returns = []
        tickers = []

        for ticker in price_candles.Ticker.unique():

            ticker_returns = (price_candles[price_candles.Ticker==ticker]['close'].values[1:]/\
                                 price_candles[price_candles.Ticker==ticker]['close'].values[:-1]) - 1
            ticker_names = [ticker for i in range(len(ticker_returns))]

            returns.append(ticker_returns)
            tickers.append(ticker_names)

        # concatenate returns and add noise
        returns = np.concatenate(returns)
        if noise:
            returns[returns==0] = np.random.normal(0,1,sum(returns==0))/1000000

        tickers = np.concatenate(tickers)
        
        if label_split == []:
            # equal-sized bins according to n_classes
            _, splits = pd.qcut(returns, q=n_classes, labels=False, retbins=True)
        elif label_split != []:
            _, splits = pd.qcut(returns, q=label_split, labels=False, retbins=True)
            
        #print(splits)

        returns = pd.DataFrame({'returns': returns, 'Ticker': tickers})

    keepCheck = []

    for ticker_iter, ticker_name in enumerate(all_features.ticker.unique()):
        ticker_features = all_features[all_features.ticker==ticker_name].copy(deep=True)
        ticker_indices = dailyIndices[dailyIndices.ticker==ticker_name].copy(deep=True)
        # removing the "ticker" variable from ticker_features as np.isnan() does not like non-numericals
        #ticker_features = ticker_features.iloc[:, ticker_features.columns != 'ticker']
        ticker_features.drop('ticker', axis=1, inplace=True)
        # extract first 4 columns as the lag0 or raw OHLC prices (used for labelling)
        #ticker_prices = price_candles[price_candles.Ticker==ticker_name]['close'].values # candles.iloc[:, :4].values
        ticker_returns = returns[returns.Ticker==ticker_name]['returns'].values

        if not safe_burn_in:
            assert data_sample == 'full'
            # we assume data_sample is full and that we can continue features from yesterday's values.
            # that we have a single burn-in at the beginning and that's it

            # get first index that has no NaNs (the sum checks for True across columns, we look for sum == 0 and where that is first True)
            burned_in_idx = np.where((np.sum(np.isnan(ticker_features.values), axis=1) == 0) == True)[0][0]
            keepCheck.append(burned_in_idx)
            # calculate end-point cut-off to match with labels
            end_point_cut = max(prediction_horizon, n_feature_lags + 1)

            # slice away the observations used for burn-in (taking off 1 at the end to match with labels [slice off "prediction_horizon"])
            burned_in_features = ticker_features.iloc[burned_in_idx : -end_point_cut, :] #.reset_index(drop=True) # features[burned_in_idx:] latter is sligthly faster but maybe not as precise
            burned_in_indices = ticker_indices.iloc[burned_in_idx : -end_point_cut, :]
            # slice away the burned-in indices from labels
            labels = extract_labels_multi_v6(data = ticker_returns[(burned_in_idx+n_feature_lags):],
                                                classes = n_classes,
                                                splits = splits)
            # labels, returns, thresholds = extract_labels(data = candles[burned_in_idx + n_feature_lags : , :],
            #                                             classes = n_classes, group_style = 'equal')

            # check if there are remaining NaNs are burn-in (means error)
            remaining_nans = np.where(np.isnan(burned_in_features.values))[0].size
            if remaining_nans > 0:
                raise ValueError('Had NaN in burned_in_features after burn-in')

        # Adding the ticker
        burned_in_features.loc[:,'ticker'] = ticker_name

        # Adding the burned in data
        all_burned_in_features = pd.concat([all_burned_in_features, burned_in_features.reset_index(drop=True)])
        all_burned_in_indices = pd.concat([all_burned_in_indices, burned_in_indices.reset_index(drop=True)])
        all_labels = pd.concat([all_labels, pd.Series(labels)])
        print(ticker_name + " done")

    # Returning the ticker as dummies
    if ticker_dummies:

        tickers = all_burned_in_features.pop('ticker')
        all_burned_in_features = pd.concat([all_burned_in_features, pd.get_dummies(tickers, prefix='d_ticker', drop_first=False)], axis=1)
#     print('Are all burned_in_idx the same?', all(keepCheck==keepCheck[0]))
#     print(dailyIndicies.head(50))
    return all_burned_in_features.reset_index(drop=True),\
            all_labels.reset_index(drop=True),\
            all_burned_in_indices.reset_index(drop=True)

In [67]:
########### Generate Labels ################

n_classes = 5
# extract first 4 columns as the lag0 or raw OHLC prices (used for labelling)
price_candles = data[['open','high','low','close','Ticker']]

########### Align Data ################

# from imported function (see testing_preprocessing_features_and_labels.ipynb for thorough experimenting with all the cut-offs):    
X_new, y_new, indices = align_features_and_labels_multi_v8(price_candles = price_candles, 
                                                         all_features = features,
                                                         prediction_horizon = 1, 
                                                         n_feature_lags = n_feature_lags, 
                                                         n_classes = n_classes, # 5,
                                                         label_split = [0,0.1,0.3,0.7,0.9,1], # empty means equal, otherwise specify sorted quantile splits including 0 and 1
                                                         safe_burn_in = False, 
                                                         data_sample = 'full',
                                                         splitType='global',
                                                         noise=False,
                                                         ticker_dummies=False)

AAPL done
ABBV done
ABT done
AEP done
AMT done
APD done
BA done
BABA done
BAC done
BHP done
BP done
CCI done
CHL done
COST done
CSGP done
D done
DIS done
ECL done
ENB done
EXC done
FB done
FMX done
GOOG done
INTC done
JNJ done
KO done
LFC done
LIN done
LMT done
MA done
MCD done
MSFT done
NKE done
NVDA done
NVS done
Number of NaNs in label: 1. 1 is expected
Returns that lead to NaNs in label: [0.0907158]
PBR done
PEP done
PFE done
PLD done
PSA done
PTR done
PYPL done
RTX done
SHW done
SNP done
SO done
SRE done
T done
TM done
TSLA done
TSM done
UNP done
UPS done
V done
WMT done


In [60]:
y[0].value_counts()

1.0    234696
0.0    194194
Name: 0, dtype: int64

In [59]:
y_new[0].value_counts()

1.0    234696
0.0    194194
Name: 0, dtype: int64

In [62]:
# test 3 class equal splits:
y_new[0].value_counts()

1.0    142979
0.0    142957
2.0    142954
Name: 0, dtype: int64

In [66]:
# test 3 class 20-60-20 splits:
y_new[0].value_counts()

1.0    257354
0.0     85772
2.0     85764
Name: 0, dtype: int64

In [64]:
# test 5 class equal splits:
y_new[0].value_counts()

1.0    85788
3.0    85783
2.0    85783
0.0    85772
4.0    85764
Name: 0, dtype: int64

In [68]:
# test 5 class 10-20-40-20-10 splits:
y_new[0].value_counts()

2.0    171567
1.0     85785
3.0     85783
0.0     42882
4.0     42873
Name: 0, dtype: int64

In [43]:
X

,open_lag0,high_lag0,low_lag0,close_lag0,spread_open_lag0,spread_high_lag0,spread_low_lag0,spread_close_lag0,bidsize_open_lag0,bidsize_high_lag0,...,sector_Consumer Cyclical,sector_Consumer Defensive,sector_Energy,sector_Financial Services,sector_Healthcare,sector_Industrials,sector_Real Estate,sector_Technology,sector_Utilities,ticker
0,0.240,0.330,-0.655,289.020,0.24,0.45,0.01,0.10,9.0,20.0,...,0,0,0,0,0,0,0,1,0,AAPL
1,0.455,1.125,-0.300,288.580,0.07,0.49,0.01,0.30,1.0,50.0,...,0,0,0,0,0,0,0,1,0,AAPL
2,-0.610,0.220,-0.815,289.095,0.49,0.49,0.01,0.17,1.0,25.0,...,0,0,0,0,0,0,0,1,0,AAPL
3,-1.220,0.115,-1.380,290.320,0.16,0.33,0.01,0.10,13.0,71.0,...,0,0,0,0,0,0,0,1,0,AAPL
4,0.235,0.355,-0.565,290.085,0.10,0.42,0.01,0.05,2.0,86.0,...,0,0,0,0,0,0,0,1,0,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428885,-0.110,0.010,-0.175,123.950,0.02,0.07,0.01,0.02,3.0,6.0,...,0,1,0,0,0,0,0,0,0,WMT
428886,-0.150,0.010,-0.190,124.100,0.02,0.07,0.01,0.04,1.0,11.0,...,0,1,0,0,0,0,0,0,0,WMT
428887,0.090,0.090,-0.075,123.995,0.01,0.06,0.01,0.01,1.0,8.0,...,0,1,0,0,0,0,0,0,0,WMT
428888,-0.340,0.020,-0.350,124.335,0.01,0.07,0.01,0.05,4.0,16.0,...,0,1,0,0,0,0,0,0,0,WMT


In [42]:
X_new

,open_lag0,high_lag0,low_lag0,close_lag0,spread_open_lag0,spread_high_lag0,spread_low_lag0,spread_close_lag0,bidsize_open_lag0,bidsize_high_lag0,...,sector_Consumer Cyclical,sector_Consumer Defensive,sector_Energy,sector_Financial Services,sector_Healthcare,sector_Industrials,sector_Real Estate,sector_Technology,sector_Utilities,ticker
0,0.240,0.330,-0.655,289.020,0.24,0.45,0.01,0.10,9.0,20.0,...,0,0,0,0,0,0,0,1,0,AAPL
1,0.455,1.125,-0.300,288.580,0.07,0.49,0.01,0.30,1.0,50.0,...,0,0,0,0,0,0,0,1,0,AAPL
2,-0.610,0.220,-0.815,289.095,0.49,0.49,0.01,0.17,1.0,25.0,...,0,0,0,0,0,0,0,1,0,AAPL
3,-1.220,0.115,-1.380,290.320,0.16,0.33,0.01,0.10,13.0,71.0,...,0,0,0,0,0,0,0,1,0,AAPL
4,0.235,0.355,-0.565,290.085,0.10,0.42,0.01,0.05,2.0,86.0,...,0,0,0,0,0,0,0,1,0,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428885,-0.110,0.010,-0.175,123.950,0.02,0.07,0.01,0.02,3.0,6.0,...,0,1,0,0,0,0,0,0,0,WMT
428886,-0.150,0.010,-0.190,124.100,0.02,0.07,0.01,0.04,1.0,11.0,...,0,1,0,0,0,0,0,0,0,WMT
428887,0.090,0.090,-0.075,123.995,0.01,0.06,0.01,0.01,1.0,8.0,...,0,1,0,0,0,0,0,0,0,WMT
428888,-0.340,0.020,-0.350,124.335,0.01,0.07,0.01,0.05,4.0,16.0,...,0,1,0,0,0,0,0,0,0,WMT


## Splitting the data

In [5]:
# Let's have a proper split (along tickers & dates)
train_size = 0.8
data_splits = pd.DataFrame()
data_splits = X.index.to_series().groupby(X['ticker']).agg(['first','last']).reset_index()

data_splits['val_size'] = ((1-train_size) * (data_splits['last'] - data_splits['first'])).astype(int)
data_splits['val_start_idx'] = data_splits['last'] - data_splits['val_size']
data_splits['val_end_idx'] = data_splits['last'] + 1 # to get the last observation included

data_splits['train_start_idx'] =  data_splits['first']
data_splits['train_end_idx'] = data_splits['val_start_idx']

# Store ranges

train_ranges = [list(x) for x in zip(data_splits['train_start_idx'], data_splits['train_end_idx'])]
val_ranges = [list(x) for x in zip(data_splits['val_start_idx'], data_splits['val_end_idx'])]


if verbose:
    data_splits

INFO:MainThread:numexpr.utils:NumExpr defaulting to 4 threads.


## Adding ticker dummies

In [6]:
## Adding ticker dummies
tickers = X.pop('ticker')
X = pd.concat([X, pd.get_dummies(tickers, prefix='ticker', drop_first=False)], axis=1)

In [13]:
X.columns

Index(['open_lag0', 'high_lag0', 'low_lag0', 'close_lag0', 'spread_open_lag0',
       'spread_high_lag0', 'spread_low_lag0', 'spread_close_lag0',
       'bidsize_open_lag0', 'bidsize_high_lag0', 'bidsize_low_lag0',
       'bidsize_close_lag0', 'ofrsize_open_lag0', 'ofrsize_high_lag0',
       'ofrsize_low_lag0', 'ofrsize_close_lag0', 'open_lag1', 'high_lag1',
       'low_lag1', 'close_lag1', 'spread_open_lag1', 'spread_high_lag1',
       'spread_low_lag1', 'spread_close_lag1', 'bidsize_open_lag1',
       'bidsize_high_lag1', 'bidsize_low_lag1', 'bidsize_close_lag1',
       'ofrsize_open_lag1', 'ofrsize_high_lag1', 'ofrsize_low_lag1',
       'ofrsize_close_lag1', 'sector_Basic Materials',
       'sector_Communication Services', 'sector_Consumer Cyclical',
       'sector_Consumer Defensive', 'sector_Energy',
       'sector_Financial Services', 'sector_Healthcare', 'sector_Industrials',
       'sector_Real Estate', 'sector_Technology', 'sector_Utilities',
       'ticker_AAPL', 'ticker_ABBV

In [21]:
train_ranges[0]

[0, 6238]

## Constructing our final train/validation sets

In [39]:
train_ds = pd.concat([X.iloc[start:end, :] for (start, end) in train_ranges]).reset_index(drop=True)
train_y = pd.concat([y.iloc[start:end] for (start, end) in train_ranges]).reset_index(drop=True)

validate_ds = pd.concat([X.iloc[start:end, :] for (start, end) in val_ranges]).reset_index(drop=True)
val_y = pd.concat([y.iloc[start:end] for (start, end) in val_ranges]).reset_index(drop=True)

train_ds.shape, train_y.shape, validate_ds.shape, val_y.shape, train_y.shape[0] + val_y.shape[0]

((343090, 98), (343090, 1), (85800, 98), (85800, 1), 428890)

In [40]:
train_ranges[0]

[0, 6238]

In [41]:
train_ranges[0][1]+10

6248

## Pre-processing

In [44]:
{i:colname for i,colname in enumerate(train_ds.columns)}

{0: 'open_lag0',
 1: 'high_lag0',
 2: 'low_lag0',
 3: 'close_lag0',
 4: 'spread_open_lag0',
 5: 'spread_high_lag0',
 6: 'spread_low_lag0',
 7: 'spread_close_lag0',
 8: 'bidsize_open_lag0',
 9: 'bidsize_high_lag0',
 10: 'bidsize_low_lag0',
 11: 'bidsize_close_lag0',
 12: 'ofrsize_open_lag0',
 13: 'ofrsize_high_lag0',
 14: 'ofrsize_low_lag0',
 15: 'ofrsize_close_lag0',
 16: 'open_lag1',
 17: 'high_lag1',
 18: 'low_lag1',
 19: 'close_lag1',
 20: 'spread_open_lag1',
 21: 'spread_high_lag1',
 22: 'spread_low_lag1',
 23: 'spread_close_lag1',
 24: 'bidsize_open_lag1',
 25: 'bidsize_high_lag1',
 26: 'bidsize_low_lag1',
 27: 'bidsize_close_lag1',
 28: 'ofrsize_open_lag1',
 29: 'ofrsize_high_lag1',
 30: 'ofrsize_low_lag1',
 31: 'ofrsize_close_lag1',
 32: 'sector_Basic Materials',
 33: 'sector_Communication Services',
 34: 'sector_Consumer Cyclical',
 35: 'sector_Consumer Defensive',
 36: 'sector_Energy',
 37: 'sector_Financial Services',
 38: 'sector_Healthcare',
 39: 'sector_Industrials',
 40: 

In [47]:
# Creating one ppdict for individual preprocessings
# ppdict1 = {'open':'minmax',
#           'high':'log',
#           'low':'log',
#           'close':'std'}
splitpoint = 32

# Standardize some features
ppdict1 = {i:'std' for i in train_ds.columns[0:splitpoint]} 
# Keep some in actual levels (Dummies in this case).
ppdict2 = {i:'act' for i in train_ds.columns[splitpoint:]} 

# Merging the two
ppdict = {**ppdict1,**ppdict2}

In [54]:
train_ds,validate_ds = pre_processing(train_ds,
                                    validate_ds,
                                    ppdict,
                                    100,
                                    verbose =True)

Pre-Processing Procedure:  act
Columns Processed: ['sector_Basic Materials' 'sector_Communication Services'
 'sector_Consumer Cyclical' 'sector_Consumer Defensive' 'sector_Energy'
 'sector_Financial Services' 'sector_Healthcare' 'sector_Industrials'
 'sector_Real Estate' 'sector_Technology' 'sector_Utilities' 'ticker_AAPL'
 'ticker_ABBV' 'ticker_ABT' 'ticker_AEP' 'ticker_AMT' 'ticker_APD'
 'ticker_BA' 'ticker_BABA' 'ticker_BAC' 'ticker_BHP' 'ticker_BP'
 'ticker_CCI' 'ticker_CHL' 'ticker_COST' 'ticker_CSGP' 'ticker_D'
 'ticker_DIS' 'ticker_ECL' 'ticker_ENB' 'ticker_EXC' 'ticker_FB'
 'ticker_FMX' 'ticker_GOOG' 'ticker_INTC' 'ticker_JNJ' 'ticker_KO'
 'ticker_LFC' 'ticker_LIN' 'ticker_LMT' 'ticker_MA' 'ticker_MCD'
 'ticker_MSFT' 'ticker_NKE' 'ticker_NVDA' 'ticker_NVS' 'ticker_PBR'
 'ticker_PEP' 'ticker_PFE' 'ticker_PLD' 'ticker_PSA' 'ticker_PTR'
 'ticker_PYPL' 'ticker_RTX' 'ticker_SHW' 'ticker_SNP' 'ticker_SO'
 'ticker_SRE' 'ticker_T' 'ticker_TM' 'ticker_TSLA' 'ticker_TSM'
 'ticker_UNP' 't

In [53]:
ppX_train.iloc[:,0].mean(),ppX_train.iloc[:,0].std()

(-1.8927265537610815e-16, 1.000001457346533)

## Prepping for models

In [8]:
N_VALIDATION = val_y.shape[0] #int(1e3)
N_TRAIN = train_y.shape[0] #int(1e4)
# BUFFER_SIZE = int(1e4)
BATCH_SIZE = 256 #512 #32
MAX_EPOCHS = 500

STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

N_REPEAT = int(N_TRAIN / ((STEPS_PER_EPOCH * MAX_EPOCHS) / BATCH_SIZE))
FEATURES = X.shape[1]

N_TRAIN, N_VALIDATION, N_TRAIN + N_VALIDATION, STEPS_PER_EPOCH, N_REPEAT, STEPS_PER_EPOCH * MAX_EPOCHS

(343090, 85800, 428890, 1340, 131, 670000)

## A Logistic Regression model in TF/Keras

In [55]:
METRICS = [
      #keras.metrics.TruePositives(name='tp'),
      #keras.metrics.FalsePositives(name='fp'),
      #keras.metrics.TrueNegatives(name='tn'),
      #keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      #keras.metrics.Precision(name='precision'),
      #keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

# def make_model(metrics = METRICS, output_bias=None):
#   if output_bias is not None:
#     output_bias = tf.keras.initializers.Constant(output_bias)
#   model = keras.Sequential([
#       keras.layers.Dense(
#           16, activation='relu',
#           input_shape=(train_features.shape[-1],)),
#       keras.layers.Dropout(0.5),
#       keras.layers.Dense(1, activation='sigmoid',
#                          bias_initializer=output_bias),
#   ])

#   model.compile(
#       optimizer=keras.optimizers.Adam(lr=1e-3),
#       loss=keras.losses.BinaryCrossentropy(),
#       metrics=metrics)

#   return model

# model = keras.Sequential({
#   keras.layers.Dense(1, input_shape=(FEATURES,))
# })

model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(28, 28)),
#     keras.layers.Dense(128, activation='relu'),
#     keras.layers.Dense(10)
    keras.layers.Dense(1,
                       input_shape=(FEATURES,),
                       activation='sigmoid',
                       kernel_regularizer=regularizers.l2(1))
])

model.summary()

# with final activation (Keras/TF tutorial advises against this practice, but they also use it later in the tutorial)
# model = keras.Sequential({
#   keras.layers.Dense(1, input_shape=(FEATURES,), activation='sigmoid')
# })

#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy', ])
model.compile(
              optimizer=keras.optimizers.Adam(), #lr=1e-3
              loss=keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=METRICS)

early_stopping = tf.keras.callbacks.EarlyStopping(
                                                monitor='val_auc', 
                                                verbose=1,
                                                patience=100,
                                                mode='max',
                                                restore_best_weights=True)

def get_callbacks(run_id):
      return [
             tfdocs.modeling.EpochDots(),
             early_stopping,
             tf.keras.callbacks.TensorBoard(logdir), #/run_id),
      ]

baseline_history = model.fit(
                            train_ds, #train_features,
                            train_y, #train_labels,
                            batch_size=512, #BATCH_SIZE,
                            epochs=1000, #EPOCHS,
                            callbacks = get_callbacks(run_id = 'first'), #[early_stopping],
                            validation_data=(validate_ds, val_y),
                            verbose=0) #(val_features, val_labels))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 99        
Total params: 99
Trainable params: 99
Non-trainable params: 0
_________________________________________________________________



Epoch: 0, accuracy:0.5352,  auc:0.5034,  loss:0.8996,  val_accuracy:0.5456,  val_auc:0.5453,  val_loss:0.6876,  
....................................................................................................
Epoch: 100, accuracy:0.5480,  auc:0.5440,  loss:0.6873,  val_accuracy:0.5454,  val_auc:0.5459,  val_loss:0.6879,  
..................Restoring model weights from the end of the best epoch.
Epoch 00118: early stopping


In [56]:
model.evaluate(validate_ds,  val_y, verbose=2)

2682/2682 - 6s - loss: 0.6879 - accuracy: 0.5457 - auc: 0.5513


[0.6878659725189209, 0.5456876754760742, 0.5513222217559814]

In [11]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [12]:
import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
%tensorboard --logdir logs

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 9296.